In [117]:
import os

In [118]:
import pandas as pd
import numpy as np

In [119]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [120]:
from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy


In [121]:
engine = create_engine("sqlite:///db/DACA.sqlite")
engine

Engine(sqlite:///db/DACA.sqlite)

In [122]:
app = Flask(__name__)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/DACA.sqlite"
db = SQLAlchemy(app)

C:\Users\kmcle\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [123]:
db

<SQLAlchemy engine=sqlite:///C:\Users\kmcle\Desktop\Project 2\Testing\db/DACA.sqlite>

In [124]:
# con.execute('alter table my_table drop id')
# con.execute('alter table my_table add id serial primary key')

In [125]:
# reflect an existing database into a new model
Base = automap_base()

    
# reflect the tables
Base.prepare(engine, reflect=True)

In [126]:
Ages = Base.classes.Age
Sexes = Base.classes.Sex
Base.classes.keys()

['Age', 'Sex']

In [127]:
session = Session(engine)


In [128]:
inspector = inspect(engine)

In [129]:
inspector.get_table_names()

['Age',
 'CBSA',
 'Country_Of_Birth',
 'Marital_Status',
 'Sex',
 'State_of_residence',
 '_source_info_',
 'coordinates',
 'sqlite_sequence']

In [130]:
Ages_columns = inspector.get_columns('Age')
for c in Ages_columns:
    print(c['name'], c["type"])

Age TEXT
Number INTEGER
Percent REAL
ID INTEGER


In [131]:
# session.query(Age.Number).all()
Totals = engine.execute('SELECT Number FROM Age').fetchall()

In [132]:
Totals

[(689800,), (2000,), (196500,), (253100,), (163400,), (74900,), (5,)]

In [133]:
session.query(Ages.Age,Ages.Number,Ages.Percent).all()

[('Total', 689800, 100.0),
 ('Under 16', 2000, 0.3),
 ('16-20', 196500, 28.5),
 ('21-25', 253100, 36.7),
 ('26-30', 163400, 23.7),
 ('31-36', 74900, 10.9),
 ('Not available', 5, 0.0)]

In [134]:
Ages_df = pd.DataFrame(session.query(Ages.Age,Ages.Number,Ages.Percent).all())
Ages_df

,Age,Number,Percent
0,Total,689800,100.0
1,Under 16,2000,0.3
2,16-20,196500,28.5
3,21-25,253100,36.7
4,26-30,163400,23.7
5,31-36,74900,10.9
6,Not available,5,0.0


In [135]:
stmt = db.session.query(Ages).statement
stmt
df = pd.read_sql_query(stmt, db.session.bind)
df

,Age,Number,Percent,ID
0,Total,689800,100.0,1
1,Under 16,2000,0.3,2
2,16-20,196500,28.5,3
3,21-25,253100,36.7,4
4,26-30,163400,23.7,5
5,31-36,74900,10.9,6
6,Not available,5,0.0,7


In [139]:
tableList = inspector.get_table_names()

tableList


['Age',
 'CBSA',
 'Country_Of_Birth',
 'Marital_Status',
 'Sex',
 'State_of_residence',
 '_source_info_',
 'coordinates',
 'sqlite_sequence']

In [137]:
tableList = pd.DataFrame(tableList)


In [138]:
tableList.columns=["listing"]
